In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, plot_confusion_matrix, plot_roc_curve
from sklearn.tree import plot_tree

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
crashes = pd.read_csv("data/Traffic_Crashes_-_Crashes.csv")

In [3]:
people = pd.read_csv("data/Traffic_Crashes_-_People.csv")

In [4]:
vehicles = pd.read_csv("data/Traffic_Crashes_-_Vehicles.csv") 

In [5]:
crashes.shape

(684836, 49)

In [6]:
people.shape

(1501548, 30)

In [7]:
vehicles.shape

(1395613, 72)

In [8]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684836 entries, 0 to 684835
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                684836 non-null  object 
 1   RD_NO                          681502 non-null  object 
 2   CRASH_DATE_EST_I               51964 non-null   object 
 3   CRASH_DATE                     684836 non-null  object 
 4   POSTED_SPEED_LIMIT             684836 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         684836 non-null  object 
 6   DEVICE_CONDITION               684836 non-null  object 
 7   WEATHER_CONDITION              684836 non-null  object 
 8   LIGHTING_CONDITION             684836 non-null  object 
 9   FIRST_CRASH_TYPE               684836 non-null  object 
 10  TRAFFICWAY_TYPE                684836 non-null  object 
 11  LANE_CNT                       198997 non-null  float64
 12  ALIGNMENT                     

### Dropping Unneccessary Columns

In [9]:
#dropping columns either not relevant, or duplicate.
crashes_drop = crashes[['CRASH_DATE', 'CRASH_HOUR','RD_NO', 
                        'CRASH_DATE_EST_I', 'DATE_POLICE_NOTIFIED','TRAFFIC_CONTROL_DEVICE',
                        'DEVICE_CONDITION', 'STREET_NO',
                        'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE',
                        'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 
                        'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I',
                        'LANE_CNT','REPORT_TYPE', 'CRASH_MONTH','LATITUDE',
                        'LONGITUDE','LOCATION','INJURIES_TOTAL','INJURIES_FATAL', 
                        'INJURIES_INCAPACITATING','INJURIES_NON_INCAPACITATING',
                        'INJURIES_REPORTED_NOT_EVIDENT','INJURIES_NO_INDICATION', 
                        'INJURIES_UNKNOWN','CRASH_MONTH','TRAFFICWAY_TYPE',
                        'NOT_RIGHT_OF_WAY_I','HIT_AND_RUN_I','INTERSECTION_RELATED_I']]

In [10]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1501548 entries, 0 to 1501547
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1501548 non-null  object 
 1   PERSON_TYPE            1501548 non-null  object 
 2   CRASH_RECORD_ID        1501548 non-null  object 
 3   RD_NO                  1494425 non-null  object 
 4   VEHICLE_ID             1471946 non-null  float64
 5   CRASH_DATE             1501548 non-null  object 
 6   SEAT_NO                303980 non-null   float64
 7   CITY                   1096629 non-null  object 
 8   STATE                  1111104 non-null  object 
 9   ZIPCODE                1002176 non-null  object 
 10  SEX                    1478107 non-null  object 
 11  AGE                    1065042 non-null  float64
 12  DRIVERS_LICENSE_STATE  882647 non-null   object 
 13  DRIVERS_LICENSE_CLASS  747164 non-null   object 
 14  SAFETY_EQUIPMENT  

In [11]:
 #dropping columns either not relevant, or duplicate.
people_drop = people[['RD_NO', 'CRASH_DATE', 'CITY', 'STATE', 'ZIPCODE', 'DRIVERS_LICENSE_STATE', 
                      'DRIVERS_LICENSE_CLASS','HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO',
                      'DRIVER_ACTION', 'DRIVER_VISION','PHYSICAL_CONDITION', 'PEDPEDAL_ACTION',
                      'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'BAC_RESULT', 
                      'BAC_RESULT VALUE', 'CELL_PHONE_USE']]

In [12]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1395613 entries, 0 to 1395612
Data columns (total 72 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   CRASH_UNIT_ID             1395613 non-null  int64  
 1   CRASH_RECORD_ID           1395613 non-null  object 
 2   RD_NO                     1388852 non-null  object 
 3   CRASH_DATE                1395613 non-null  object 
 4   UNIT_NO                   1395613 non-null  int64  
 5   UNIT_TYPE                 1393686 non-null  object 
 6   NUM_PASSENGERS            206673 non-null   float64
 7   VEHICLE_ID                1363941 non-null  float64
 8   CMRC_VEH_I                26052 non-null    object 
 9   MAKE                      1363936 non-null  object 
 10  MODEL                     1363792 non-null  object 
 11  LIC_PLATE_STATE           1240091 non-null  object 
 12  VEHICLE_YEAR              1142136 non-null  float64
 13  VEHICLE_DEFECT            1

In [13]:
#dropping columns either not relevant, or duplicate.
vehicles_drop = vehicles[['RD_NO', 'CRASH_DATE', 'LIC_PLATE_STATE', 'AREA_00_I', 'AREA_01_I', 
                                       'AREA_02_I', 'AREA_03_I','AREA_04_I','AREA_05_I','AREA_06_I','AREA_07_I',
                                       'AREA_08_I','AREA_09_I','AREA_10_I','AREA_11_I','AREA_12_I','AREA_99_I',
                                       'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO', 'COMMERCIAL_SRC', 'GVWR',
                                       'CARRIER_NAME', 'CARRIER_CITY', 'HAZMAT_PLACARDS_I',
                                       'HAZMAT_NAME', 'UN_NO','HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 
                                       'HAZMAT_REPORT_NO', 'MCS_REPORT_I', 'IDOT_PERMIT_NO',
                                       'WIDE_LOAD_I', 'TRAILER1_WIDTH', 'TRAILER2_WIDTH', 'TRAILER1_LENGTH', 
                                       'TRAILER2_LENGTH', 'TOTAL_VEHICLE_LENGTH', 'AXLE_CNT',
                                       'LOAD_TYPE', 'HAZMAT_OUT_OF_SERVICE_I', 'MCS_OUT_OF_SERVICE_I', 
                                       'HAZMAT_CLASS', 'MAKE', 'MODEL', 'VEHICLE_YEAR', 'VEHICLE_DEFECT', 
                                       'VEHICLE_CONFIG', 'CARGO_BODY_TYPE', 'TOWED_BY', 
                                       'TOWED_I', 'CARRIER_STATE',
                                       'FIRE_I', 'MCS_REPORT_NO','HAZMAT_VIO_CAUSE_CRASH_I',
                                       'MCS_VIO_CAUSE_CRASH_I', 'TOWED_TO',
                                       'CMRC_VEH_I','VEHICLE_ID', 'OCCUPANT_CNT', 'NUM_PASSENGERS',
                                       'TRAVEL_DIRECTION', 'MANEUVER',
                                       'VEHICLE_USE', 'CRASH_UNIT_ID', 'UNIT_TYPE']]

In [14]:
 #dropping columns either not relevant, or duplicate.
crashes.drop(columns=crashes_drop, inplace=True)
vehicles.drop(columns=vehicles_drop, inplace=True)
people.drop(columns=people_drop, inplace=True)

### Merging Datasets on the Crash_Record_ID Column

In [30]:
#Merging Datasets on the Crash_Record_ID Column
merge = pd.merge(crashes, vehicles, on='CRASH_RECORD_ID')
df_merge = pd.merge(merge, people, on='CRASH_RECORD_ID')

### Previewing shape, data and info 

In [31]:
# previewing shape, data and info 
df_merge.head()

,CRASH_RECORD_ID,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,DAMAGE,...,PERSON_ID,PERSON_TYPE,VEHICLE_ID,SEAT_NO,SEX,AGE,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,INJURY_CLASSIFICATION
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,30,CLEAR,DAYLIGHT,TURNING,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",...,P138006,PASSENGER,582401.0,4.0,F,27.0,SAFETY BELT USED,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY
1,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,30,CLEAR,DAYLIGHT,TURNING,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",...,O611178,DRIVER,582401.0,NaN,F,22.0,SAFETY BELT USED,DID NOT DEPLOY,NONE,"REPORTED, NOT EVIDENT"
2,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,30,CLEAR,DAYLIGHT,TURNING,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",...,O611179,DRIVER,582411.0,NaN,M,56.0,SAFETY BELT USED,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY
3,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,30,CLEAR,DAYLIGHT,TURNING,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",...,P138006,PASSENGER,582401.0,4.0,F,27.0,SAFETY BELT USED,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY
4,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,30,CLEAR,DAYLIGHT,TURNING,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",...,O611178,DRIVER,582401.0,NaN,F,22.0,SAFETY BELT USED,DID NOT DEPLOY,NONE,"REPORTED, NOT EVIDENT"


In [21]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3147856 entries, 0 to 3147855
Data columns (total 30 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   index                    int64  
 1   CRASH_RECORD_ID          object 
 2   POSTED_SPEED_LIMIT       int64  
 3   WEATHER_CONDITION        object 
 4   LIGHTING_CONDITION       object 
 5   FIRST_CRASH_TYPE         object 
 6   ALIGNMENT                object 
 7   ROADWAY_SURFACE_COND     object 
 8   ROAD_DEFECT              object 
 9   CRASH_TYPE               object 
 10  DAMAGE                   object 
 11  PRIM_CONTRIBUTORY_CAUSE  object 
 12  SEC_CONTRIBUTORY_CAUSE   object 
 13  NUM_UNITS                int64  
 14  MOST_SEVERE_INJURY       object 
 15  CRASH_DAY_OF_WEEK        int64  
 16  UNIT_NO                  int64  
 17  VEHICLE_TYPE             object 
 18  EXCEED_SPEED_LIMIT_I     object 
 19  FIRST_CONTACT_POINT      object 
 20  PERSON_ID                object 
 21  PERSON_T

### Calculating nulls

In [32]:
# calculating nulls
nulls = df_merge.isna().sum()
nulls

CRASH_RECORD_ID                  0
POSTED_SPEED_LIMIT               0
WEATHER_CONDITION                0
LIGHTING_CONDITION               0
FIRST_CRASH_TYPE                 0
ALIGNMENT                        0
ROADWAY_SURFACE_COND             0
ROAD_DEFECT                      0
CRASH_TYPE                       0
DAMAGE                           0
PRIM_CONTRIBUTORY_CAUSE          0
SEC_CONTRIBUTORY_CAUSE           0
NUM_UNITS                        0
MOST_SEVERE_INJURY              21
CRASH_DAY_OF_WEEK                0
UNIT_NO                          0
VEHICLE_TYPE                 72904
EXCEED_SPEED_LIMIT_I       3141736
FIRST_CONTACT_POINT          97428
PERSON_ID                        0
PERSON_TYPE                      0
VEHICLE_ID                   64444
SEAT_NO                    2504742
SEX                          49851
AGE                         908769
SAFETY_EQUIPMENT              9810
AIRBAG_DEPLOYED              61246
EJECTION                     40387
INJURY_CLASSIFICATIO

In [33]:
df_merge = df_merge.fillna(0)

In [26]:
df_merge.isna().sum().sumdf_merge.info()

<bound method Series.sum of index                      0
CRASH_RECORD_ID            0
POSTED_SPEED_LIMIT         0
WEATHER_CONDITION          0
LIGHTING_CONDITION         0
FIRST_CRASH_TYPE           0
ALIGNMENT                  0
ROADWAY_SURFACE_COND       0
ROAD_DEFECT                0
CRASH_TYPE                 0
DAMAGE                     0
PRIM_CONTRIBUTORY_CAUSE    0
SEC_CONTRIBUTORY_CAUSE     0
NUM_UNITS                  0
MOST_SEVERE_INJURY         0
CRASH_DAY_OF_WEEK          0
UNIT_NO                    0
VEHICLE_TYPE               0
EXCEED_SPEED_LIMIT_I       0
FIRST_CONTACT_POINT        0
PERSON_ID                  0
PERSON_TYPE                0
VEHICLE_ID                 0
SEAT_NO                    0
SEX                        0
AGE                        0
SAFETY_EQUIPMENT           0
AIRBAG_DEPLOYED            0
EJECTION                   0
INJURY_CLASSIFICATION      0
dtype: int64>

In [34]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3147856 entries, 0 to 3147855
Data columns (total 29 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CRASH_RECORD_ID          object 
 1   POSTED_SPEED_LIMIT       int64  
 2   WEATHER_CONDITION        object 
 3   LIGHTING_CONDITION       object 
 4   FIRST_CRASH_TYPE         object 
 5   ALIGNMENT                object 
 6   ROADWAY_SURFACE_COND     object 
 7   ROAD_DEFECT              object 
 8   CRASH_TYPE               object 
 9   DAMAGE                   object 
 10  PRIM_CONTRIBUTORY_CAUSE  object 
 11  SEC_CONTRIBUTORY_CAUSE   object 
 12  NUM_UNITS                int64  
 13  MOST_SEVERE_INJURY       object 
 14  CRASH_DAY_OF_WEEK        int64  
 15  UNIT_NO                  int64  
 16  VEHICLE_TYPE             object 
 17  EXCEED_SPEED_LIMIT_I     object 
 18  FIRST_CONTACT_POINT      object 
 19  PERSON_ID                object 
 20  PERSON_TYPE              object 
 21  VEHICLE_

### Inspecting Values per Column

In [35]:
for col in df_merge.columns:
    try:
        print(col, df_merge[col].value_counts(dropna=False)[:10])
    except:
        print(col, df_merge[col].value_counts())
        # If there aren't 5+ unique values for a column the first print statement
        # will throw an error for an invalid idx slice
    print('\n') # Break up the output between columns

CRASH_RECORD_ID 023f7de8d0f7e90dc072b7b88b62502f306469597481dabc2469aecc67a1026d485ba885476a3ea258a0df3cf6543b77b69cdbfabca44ab64b0fc64c00c808ef    192
c9d233e31a4f2a07733ef75f0404e75c360b30c7ee9bc45076938dc80c375578c1468bc096ecb773d2bfc71270d746d95f416a5bd6b15fbcf8707b1748693722    156
74e9d893f1df817db0fd24a7d10006941f1eda0c9e7a33c62bb2850e5a9560099b88bececab94471325d467f79f04ddf70ad92357f500c1e5765905d95278524    154
09aa8d6e2906482df6717671a532324d3667b00920b363a65face81089921576c5175803211ef1fc902662b7cb7df5e7268ab8491eb4fb4444a888fac30c888a    153
645cdd6fd3ed2f043eefdc48230464a8ad66127691a2a2781240f5c7570ff5a87aaa6cf887bc1f76e93cfcb770359fb8136d59a40d8c18b62133fec286670279    144
1829f52c1281a0396ef94692331b3dc530bc4be5a54cd55e94c24a5e5e49b800fbcf9f24dabe4c8277c8964ad05aadc89e90fd94021959d6dff5fad55480d595    138
7be311dead41c5337cbd12d40bb7be93c505303d6f1cf92e72a2b7c695ae95b472a66d9b3a6b505a0e4c2279d53acf3b6115320fcafb54d8ee1aa3d0c811e3a0    130
31ecf6862c691ff12d3856213b902c14

EJECTION NONE                  2916546
UNKNOWN                176748
0                       40387
TOTALLY EJECTED          9851
PARTIALLY EJECTED        2469
TRAPPED/EXTRICATED       1855
Name: EJECTION, dtype: int64


INJURY_CLASSIFICATION NO INDICATION OF INJURY     2866127
NONINCAPACITATING INJURY     158112
REPORTED, NOT EVIDENT         89741
INCAPACITATING INJURY         30424
FATAL                          1841
0                              1611
Name: INJURY_CLASSIFICATION, dtype: int64


